## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/77-googleq-a-train-glove-2lstm-dense/model.h5'
tokenizer_path = '/kaggle/input/77-googleq-a-train-glove-2lstm-dense/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 20000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 200
ANSWER_MAX_LEN = 200

## Test set

In [6]:
tokenizer = load(tokenizer_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
print('n unknown words (GloVe): ', len(glove_unknown_words))

n unknown words (GloVe):  3306


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.5)(title)
title = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(title)
title = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False)(title)
title_out = Dense(64, activation='relu', kernel_initializer='lecun_normal')(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = SpatialDropout1D(0.5)(body)
body = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(body)
body = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False)(body)
body_out = Dense(64, activation='relu', kernel_initializer='lecun_normal')(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = SpatialDropout1D(0.5)(answer)
answer = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(answer)
answer = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False)(answer)
answer_out = Dense(64, activation='relu', kernel_initializer='lecun_normal')(answer)


# Output
x = Concatenate()([title_out, body_out, answer_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_body_seq (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      17424600    input_title_seq[0][0]            
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.911538,0.704871,0.087434,0.723714,0.835926,0.798280,0.627778,0.590521,0.344958,...,0.885524,0.910621,0.660459,0.950560,0.960261,0.850213,0.110227,0.087071,0.790681,0.906366
1,46,0.892526,0.587190,0.002363,0.700241,0.775102,0.895573,0.544799,0.459540,0.098029,...,0.762144,0.933778,0.632980,0.965320,0.975334,0.854708,0.915984,0.150807,0.105845,0.874593
2,70,0.908458,0.731591,0.059924,0.712506,0.902403,0.863908,0.621140,0.592446,0.344646,...,0.885030,0.904751,0.649230,0.947163,0.959374,0.843974,0.071441,0.066302,0.786305,0.899840
3,132,0.859281,0.545210,0.003208,0.788415,0.778145,0.940693,0.541952,0.429150,0.121955,...,0.747645,0.939566,0.641933,0.969454,0.979560,0.868621,0.821625,0.140146,0.425628,0.891036
4,200,0.876783,0.583196,0.084377,0.696136,0.738994,0.746169,0.598108,0.538091,0.342011,...,0.799892,0.891054,0.644147,0.943091,0.946418,0.820588,0.300404,0.152680,0.673707,0.890036


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.862681,0.570978,0.051355,0.685779,0.766015,0.812161,0.567274,0.493150,0.237541,...,0.761315,0.909087,0.634594,0.949010,0.958401,0.829622,0.564408,0.159232,0.466320,0.879697
std,2812.670060,0.045173,0.106924,0.071202,0.049441,0.076378,0.119672,0.039041,0.071978,0.091987,...,0.084861,0.020051,0.014542,0.013765,0.014414,0.023156,0.296272,0.049620,0.210411,0.020089
min,39.000000,0.762305,0.377221,0.000691,0.498988,0.454584,0.332512,0.502536,0.387431,0.055294,...,0.599657,0.833646,0.595557,0.893815,0.901756,0.746231,0.011206,0.025868,0.070368,0.809091
25%,2572.000000,0.825131,0.471675,0.005562,0.655186,0.731025,0.757768,0.535008,0.438986,0.156544,...,0.683130,0.895869,0.625071,0.941705,0.949495,0.812115,0.295391,0.137858,0.277953,0.866401
50%,5093.000000,0.863469,0.571018,0.015885,0.694659,0.770616,0.859420,0.560378,0.469607,0.243784,...,0.767839,0.910531,0.633428,0.951154,0.960398,0.831526,0.670830,0.169087,0.466756,0.879720
75%,7482.000000,0.900700,0.671379,0.069262,0.719769,0.810078,0.896204,0.599230,0.539710,0.313349,...,0.834406,0.924713,0.643054,0.958899,0.969197,0.846260,0.822517,0.192787,0.652480,0.894406
max,9640.000000,0.956967,0.792935,0.336250,0.791019,0.958874,0.940693,0.667704,0.706394,0.441531,...,0.918154,0.949650,0.695286,0.975726,0.985635,0.894526,0.951473,0.267272,0.888978,0.939497
